In [1]:
import sys
import json
import re
from tqdm import tqdm

print("Python executable:", sys.executable)
print("Python version:", sys.version)


Python executable: /usr/bin/python3
Python version: 3.8.10 (default, Mar 18 2025, 20:04:55) 
[GCC 9.4.0]


In [2]:
from human_eval.data import read_problems, write_jsonl

In [3]:
problems = read_problems()
print("Number of HumanEval problems:", len(problems))

# Inspect one problem
example_task = "HumanEval/1"
print("Example task keys:", problems[example_task].keys())
print("\nPrompt:\n", problems[example_task]["prompt"])


Number of HumanEval problems: 164
Example task keys: dict_keys(['task_id', 'prompt', 'entry_point', 'canonical_solution', 'test'])

Prompt:
 from typing import List


def separate_paren_groups(paren_string: str) -> List[str]:
    """ Input to this function is a string containing multiple groups of nested parentheses. Your goal is to
    separate those group into separate strings and return the list of those.
    Separate groups are balanced (each open brace is properly closed) and not nested within each other
    Ignore any spaces in the input string.
    >>> separate_paren_groups('( ) (( )) (( )( ))')
    ['()', '(())', '(()())']
    """



# Helper fn

In [4]:
import re
import math
import time
import tempfile
import subprocess
import requests
from pathlib import Path
from typing import List, Dict


In [5]:
def get_failed_results(file_path):
    """
    Reads a JSONL results file and extracts all entries where 'passed' is False.
    """
    failed_results = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line.strip())
            if not data.get("passed", False):
                failed_results.append(data)
    return failed_results


def extract_clean_code(generated_text: str) -> str:
    """
    Extracts and cleans Python code from an AI-generated response.
    """
    code_blocks = re.findall(r"```(?:python)?\s*(.*?)\s*```", generated_text, re.DOTALL)
    if code_blocks:
        return max(code_blocks, key=len).strip()
    
    lines = generated_text.splitlines()
    code_started = False
    code_lines = []
    for line in lines:
        if not code_started and line.lstrip().startswith(('def ', 'class ', 'import ', 'from ')):
            code_started = True
        if code_started:
            code_lines.append(line)
    if code_lines:
        return "\n".join(code_lines).strip()
    
    return generated_text.strip()


In [6]:
# def strip_function_def(prompt: str, generated_code: str) -> str:
#     """
#     Remove function definition if model repeats it.
#     Do NOT touch indentation here.
#     """

#     lines = generated_code.splitlines()

#     # Remove repeated function signature
#     if lines and lines[0].lstrip().startswith("def "):
#         lines = lines[1:]

#     return "\n".join(lines).rstrip()
def strip_function_def(prompt: str, generated_code: str) -> str:
    """
    Robustly removes the function signature and imports to prevent nesting errors.
    """
    # 1. Extract the function name from the prompt to know what to look for
    # (Assuming standard HumanEval prompt format ending in 'def func_name(...):')
    func_name_match = re.search(r"def\s+(\w+)\s*\(", prompt)
    func_name = func_name_match.group(1) if func_name_match else None

    lines = generated_code.splitlines()
    cleaned_lines = []
    
    # 2. Skip lines until we pass the function definition
    inside_function = False
    
    for line in lines:
        # If we see the function definition again, ignore it and toggle flag
        if func_name and f"def {func_name}" in line:
            inside_function = True
            continue # Skip the signature line itself
        
        # If we haven't found the def yet, and the line is an import or comment, keep it
        # BUT standard HumanEval usually expects indented code immediately.
        # Safer strategy: Only keep lines that are indented (the body)
        if line.startswith(" ") or line.startswith("\t"):
            cleaned_lines.append(line)
            
    # Fallback: if the sophisticated strip failed, use the simple one or raw
    if not cleaned_lines:
        return generated_code

    return "\n".join(cleaned_lines)


In [8]:
def normalize_indentation(code: str, indent: int = 4) -> str:
    """ Normalize indentation of a model-generated function body.
    - Removes excessive leading indentation
    - Preserves relative indentation
    - Re-indents everything with indent spaces (default: 4)
    """
    if not code or not code.strip():
        return ""

    lines = code.splitlines()

    # Remove empty leading/trailing lines
    while lines and not lines[0].strip():
        lines.pop(0)
    while lines and not lines[-1].strip():
        lines.pop()

    if not lines:
        return ""

    # Compute minimum indentation (ignore empty lines)
    min_indent = min(
        len(line) - len(line.lstrip())
        for line in lines
        if line.strip()
    )

    # Dedent and re-indent
    normalized = []
    base = " " * indent
    for line in lines:
        if line.strip():
            normalized.append(base + line[min_indent:])
        else:
            normalized.append("")

    return "\n".join(normalized)

import time
import requests


# Basic Code Generation


In [8]:
import time
import requests

def generate_one_completion_basic(prompt: str, model: str) -> str:
    """
    Uses local Ollama to generate a HumanEval-compatible completion.
    Returns ONLY the function body (indented).
    """

    url = "http://127.0.0.1:11434/api/generate"

    system_msg = (
        "You are a Python coding assistant.\n"
        "Complete the function below.\n"
        "Rules:\n"
        "- Output ONLY Python code\n"
        "- Do NOT repeat the function signature\n"
        "- Do NOT add comments or explanations\n"
    )

    payload = {
        "model": model,
        "prompt": system_msg + "\n\n" + prompt,
        "temperature": 0.1,
        "num_predict": 400,
        "stream": False
    }

    try:
        resp = requests.post(url, json=payload, timeout=300)
        resp.raise_for_status()
        raw = resp.json().get("response", "").strip()

        if not raw:
            raise RuntimeError("Empty response from Ollama")

        cleaned = extract_clean_code(raw)
        completion = strip_function_def(prompt, cleaned)
        return completion

    except Exception as e:
        print("[Generation error]", e)
        return "    pass"


In [8]:
from tqdm import tqdm
from human_eval.data import write_jsonl

def generate_pass_k_samples(
    problems,
    model_name: str,
    k: int
):
    """
    Generates k independent samples per task (pass@k).
    """

    samples = []

    for task_id in tqdm(problems, desc=f"Generating pass@{k} samples"):
        prompt = problems[task_id]["prompt"]

        for _ in range(k):
            completion = generate_one_completion_basic(
                prompt,
                model_name
            )

            samples.append({
                "task_id": task_id,
                "completion": completion
            })

    return samples


In [10]:
model_name = "codellama:7b"
pass_k = 1   # 🔥 change this to 1, 2, 5, 10, etc.

samples = generate_pass_k_samples(
    problems=problems,
    model_name=model_name,
    k=pass_k
)

output_file = f"samples_codellama_7b_pass@{pass_k}.jsonl"
write_jsonl(output_file, samples)

print(f"Generated samples saved to {output_file}")


Generating pass@1 samples: 100%|██████████| 164/164 [01:30<00:00,  1.82it/s]

Generated samples saved to samples_codellama_7b_pass@1.jsonl


In [11]:
! evaluate_functional_correctness samples_codellama_7b_pass@1.jsonl

Reading samples...
164it [00:00, 43188.66it/s]
Running test suites...
100%|█████████████████████████████████████████| 164/164 [00:01<00:00, 88.98it/s]
Writing results to samples_codellama_7b_pass@1.jsonl_results.jsonl...
100%|██████████████████████████████████████| 164/164 [00:00<00:00, 50008.42it/s]
{'pass@1': 0.3475609756097561}


In [29]:
from tqdm import tqdm
from human_eval.data import write_jsonl

pass_k = 1  # 🔥 change this to 1, 2, 5, 10 (pass@k)
model_name = "codeqwen:7b"

samples = []

for task_id in tqdm(problems, desc=f"Generating pass@{pass_k} ({model_name})"):
    prompt = problems[task_id]["prompt"]

    for _ in range(pass_k):
        completion = generate_one_completion_basic(
            prompt,
            model_name
        )

        samples.append({
            "task_id": task_id,
            "completion": completion
        })

output_file = f"samples_codeqwen_7b_pass@{pass_k}.jsonl"
write_jsonl(output_file, samples)

print(f"Generated samples saved to {output_file}")


Generating pass@1 (codeqwen:7b): 100%|██████████| 164/164 [02:19<00:00,  1.18it/s]

Generated samples saved to samples_codeqwen_7b_pass@1.jsonl


In [33]:
! evaluate_functional_correctness samples_codeqwen_7b_pass@1.jsonl

Reading samples...
164it [00:00, 14584.55it/s]
Running test suites...
100%|█████████████████████████████████████████| 164/164 [00:04<00:00, 36.84it/s]
Writing results to samples_codeqwen_7b_pass@1.jsonl_results.jsonl...
100%|██████████████████████████████████████| 164/164 [00:00<00:00, 47698.90it/s]
{'pass@1': 0.5670731707317073}


## SCOT_Normalized_Generation_In_One_Request


In [16]:
import requests
from tqdm import tqdm
from human_eval.data import write_jsonl

OLLAMA_URL = "http://127.0.0.1:11434/api/generate"


def ollama_generate(prompt: str, model: str, temperature=0.1, num_predict=500) -> str:
    payload = {
        "model": model,
        "prompt": prompt,
        "temperature": temperature,
        "num_predict": num_predict,
        "stream": False
    }

    resp = requests.post(OLLAMA_URL, json=payload, timeout=300)
    resp.raise_for_status()
    return resp.json().get("response", "").strip()

def generate_one_completion_SCoT(prompt: str, model: str, print_scot=False) -> str:
    """
    Normalized three-shot SCoT + Code generation (single request)
    """

    system_msg = (
        "You are a Python problem-solving and coding assistant.\n"
        "For each problem, first write a rough structured solving process "
        "in natural language using sequences, branches, and loops.\n"
        "Then write correct Python code that follows that solving process.\n"
        "The solving process may contain minor errors.\n\n"
        "Use the following strict format:\n"
        "=== Structured Solving Process ===\n"
        "<natural language steps>\n\n"
        "=== Python Code ===\n"
        "<only valid Python code>\n"
    )

    three_shot_example = """
                    === Structured Solving Process ===
                    Example 1:
                    Input: str: a string
                    Output: ch: a repeated character in str
                    1: initialize an empty set
                    2: for each character ch in str:
                    3: if ch is already in the set, return ch
                    4: otherwise add ch to the set
                    5: return None

                    === Python Code ===
                    def first_Repeated_Char(str):
                        seen = set()
                        for ch in str:
                            if ch in seen:
                                return ch
                            seen.add(ch)
                        return None


                    === Structured Solving Process ===
                    Example 2:
                    Input: nums: list[int]
                    Output: total: int
                    1: initialize total to 0
                    2: for each number n in nums:
                    3: add n * n to total
                    4: return total

                    === Python Code ===
                    def sum_of_squares(nums):
                        total = 0
                        for n in nums:
                            total += n * n
                        return total


                    === Structured Solving Process ===
                    Example 3:
                    Input: s: str
                    Output: result: bool
                    1: reverse the string s
                    2: if reversed string equals s:
                    3: return True
                    4: else return False

                    === Python Code ===
                    def is_palindrome(s):
                        return s == s[::-1]
                    """

    user_msg = f"""
                    {three_shot_example}

                    Now solve the following problem in the SAME way.

                    Problem:
                    {prompt}
                    """

    full_prompt = system_msg + "\n" + user_msg

    raw = ollama_generate(
        prompt=full_prompt,
        model=model,
        temperature=0.8,
        num_predict=600
    )

    # Optional debug print
    if print_scot:
        scot_part = raw.split("=== Python Code ===")[0]
        print("=== SCoT ===")
        print(scot_part.strip())
        print()

    # Extract code only
    if "=== Python Code ===" in raw:
        code = raw.split("=== Python Code ===")[1]
    else:
        code = raw  # fallback safety

    cleaned = extract_clean_code(code)
    return strip_function_def(prompt, cleaned)




In [21]:
# =========================
# HumanEval Sampling Loop
# =========================

pass_k = 1
model_name = "codellama:7b"

samples = []

for task_id in tqdm(problems, desc=f"Generating normalized SCoT pass@{pass_k} ({model_name})"):
    prompt = problems[task_id]["prompt"]

    for _ in range(pass_k):
        samples.append({
            "task_id": task_id,
            "completion": generate_one_completion_SCoT(
                prompt,
                model_name
            )
        })

output_file = f"samples_codellama_SCoT_normalized_pass@{pass_k}.jsonl"
write_jsonl(output_file, samples)

print(f"Generated samples saved to {output_file}")
#...

Generating normalized SCoT pass@1 (codellama:7b): 100%|██████████| 164/164 [07:34<00:00,  2.77s/it]

Generated samples saved to samples_codellama_SCoT_normalized_pass@1.jsonl


In [22]:
! evaluate_functional_correctness samples_codellama_SCoT_normalized_pass@1.jsonl

Reading samples...
164it [00:00, 28209.72it/s]
Running test suites...
100%|█████████████████████████████████████████| 164/164 [00:04<00:00, 34.37it/s]
Writing results to samples_codellama_SCoT_normalized_pass@1.jsonl_results.jsonl...
100%|██████████████████████████████████████| 164/164 [00:00<00:00, 34963.19it/s]
{'pass@1': 0.25609756097560976}


## SCOT NEW

In [9]:
import requests
from tqdm import tqdm
from human_eval.data import write_jsonl

OLLAMA_URL = "http://127.0.0.1:11434/api/generate"


# ==========================================================
# Low-level Ollama call (deterministic for pass@1)
# ==========================================================
def ollama_generate(prompt: str, model: str, temperature=0.0, num_predict=300) -> str:
    payload = {
        "model": model,
        "prompt": prompt,
        "temperature": temperature,
        "top_p": 1.0,
        "num_predict": num_predict,
        "stream": False
    }

    resp = requests.post(OLLAMA_URL, json=payload, timeout=300)
    resp.raise_for_status()
    return resp.json().get("response", "").strip()


# ==========================================================
# STEP 1 — Generate ONE SCoT (Figure 3 style)
# ==========================================================
def generate_SCoT(prompt: str, model: str) -> str:

    system_msg = (
        "You are a Python problem-solving assistant.\n"
        "Write a structured solving process.\n"
        "Do NOT write code.\n"
        "Start with Input and Output structure.\n"
        "Use only sequence, branch, and loop structures.\n"
        "Be concise. Avoid long explanations.\n"
    )

    # -------------------------
    # THREE examples (paper style)
    # -------------------------

    example_1 = """
def first_Repeated_Char(str):
    \"\"\"Find first repeated character.\"\"\"
    pass

Input: str: string
Output: ch: character or None
1: Initialize empty set
2: for each character ch in str:
3: if ch in set:
4: return ch
5: else:
6: add ch to set
7: return None
"""

    example_2 = """
def is_even_list(lst):
    \"\"\"Return True if all numbers are even.\"\"\"
    pass

Input: lst: list of integers
Output: result: boolean
1: for each number n in lst:
2: if n % 2 != 0:
3: return False
4: return True
"""

    example_3 = """
def sum_nested_list(arr):
    \"\"\"Return total sum of nested list.\"\"\"
    pass

Input: arr: list of list of integers
Output: total: integer
1: Initialize total to 0
2: for each sublist in arr:
3: for each number n in sublist:
4: total = total + n
5: return total
"""

    full_prompt = (
        system_msg
        + "\n"
        + example_1
        + "\n"
        + example_2
        + "\n"
        + example_3
        + "\nNow write the structured solving process for:\n\n"
        + prompt
    )

    return ollama_generate(
        prompt=full_prompt,
        model=model,
        temperature=0.1,
        num_predict=250
    )


# ==========================================================
# STEP 2 — Generate Code From SCoT (Figure 4 style)
# ==========================================================
def generate_code_from_scot(prompt: str, scot: str, model: str) -> str:

    system_msg = (
        "You are a Python coding assistant.\n"
        "Write Python code based on the structured solving process.\n"
        "The solving process may contain errors.\n"
        "Output ONLY valid Python code.\n"
    )

    example_code_1 = """
def first_Repeated_Char(str):
    \"\"\"Find first repeated character.\"\"\"
    Input: str: string
    Output: ch: character or None
    1: Initialize empty set
    2: for each character ch in str:
    3: if ch in set:
    4: return ch
    5: else:
    6: add ch to set
    7: return None
    \"\"\"

    seen = set()
    for ch in str:
        if ch in seen:
            return ch
        seen.add(ch)
    return None
"""

    example_code_2 = """
def is_even_list(lst):
    \"\"\"Return True if all numbers are even.\"\"\"
    Input: lst: list of integers
    Output: result: boolean
    1: for each number n in lst:
    2: if n % 2 != 0:
    3: return False
    4: return True
    \"\"\"

    for n in lst:
        if n % 2 != 0:
            return False
    return True
"""

    example_code_3 = """
def sum_nested_list(arr):
    \"\"\"Return total sum of nested list.\"\"\"
    Input: arr: list of list of integers
    Output: total: integer
    1: Initialize total to 0
    2: for each sublist in arr:
    3: for each number n in sublist:
    4: total = total + n
    5: return total
    \"\"\"

    total = 0
    for sublist in arr:
        for n in sublist:
            total += n
    return total
"""

    full_prompt = (
        system_msg
        + "\n"
        + example_code_1
        + "\n"
        + example_code_2
        + "\n"
        + example_code_3
        + "\nNow generate code for:\n\n"
        + prompt
        + "\n\nStructured Solving Process:\n"
        + scot
    )

    return ollama_generate(
        prompt=full_prompt,
        model=model,
        temperature=0.1,
        num_predict=300
    )


# ==========================================================
# MAIN LOOP — pass@1 only
# ==========================================================
pass_k = 1
model_name = "codellama:7b"

samples = []

for task_id in tqdm(problems, desc=f"SCoT pass@1 ({model_name})"):

    prompt = problems[task_id]["prompt"]

    scot = generate_SCoT(prompt, model_name)
    completion = generate_code_from_scot(prompt, scot, model_name)

    samples.append({
        "task_id": task_id,
        "completion": completion
    })

output_file = f"samples_codellama_SCoT_pass@1.jsonl"
write_jsonl(output_file, samples)

print(f"Generated samples saved to {output_file}")


SCoT pass@1 (codellama:7b):   0%|          | 0/164 [00:00<?, ?it/s]

SCoT pass@1 (codellama:7b): 100%|██████████| 164/164 [13:14<00:00,  4.84s/it]

Generated samples saved to samples_codellama_SCoT_pass@1.jsonl


In [10]:
! evaluate_functional_correctness samples_codellama_SCoT_pass@1.jsonl

Reading samples...
164it [00:00, 25742.52it/s]
Running test suites...
100%|█████████████████████████████████████████| 164/164 [00:04<00:00, 35.08it/s]
Writing results to samples_codellama_SCoT_pass@1.jsonl_results.jsonl...
100%|██████████████████████████████████████| 164/164 [00:00<00:00, 27118.70it/s]
{'pass@1': 0.024390243902439025}


# SCOT

In [7]:
import requests

OLLAMA_URL = "http://127.0.0.1:11434/api/generate"

def ollama_generate(prompt: str, model: str, temperature=0.1, num_predict=500) -> str:
    payload = {
        "model": model,
        "prompt": prompt,
        "temperature": temperature,
        "num_predict": num_predict,
        "stream": False
    }

    resp = requests.post(OLLAMA_URL, json=payload, timeout=None)
    resp.raise_for_status()

    return resp.json().get("response", "").strip()


In [8]:


def generate_SCoT(prompt: str, model: str) -> str:
    """
    One-shot SCoT generation (Figure 3 style)
    """

    system_msg = (
        "You are a Python problem-solving assistant.\n"
        "Generate a rough structured solving process in natural language.\n"
        "Do NOT write code.\n"
    )

    one_shot_example = """
        def first_Repeated_Char(str):
        \"\"\"
        Write a python function to find the first repeated
        character in a given string.
        \"\"\"
        Pass

        Please understand the requirement and write a rough solving
        process. It starts with a input-output structure. You
        should use three basic structures to build the solving
        process, including sequences, branches, and loops.
        The necessary details should be written in natural languages.

        Input: str: a string
        Output: ch: a repeated character in str
        1: for each character ch in str:
        2: if ch appears more than once in str:
        3: return ch
        4: return None
       """

    user_msg = f"""
            {one_shot_example}

            Now solve the following problem in the SAME way.

            {prompt}
            """

    full_prompt = system_msg + "\n" + user_msg

    return ollama_generate(
        prompt=full_prompt,
        model=model,
        temperature=0.1,
        num_predict=500
    )



In [9]:


# def generate_one_completion_SCoT(prompt: str, model: str, print_scot=True) -> str:
#     """
#     One-shot code generation using SCoT (Figure 4 style)
#     """

#     scot = generate_SCoT(prompt, model)
#     # print("=== SCoT ===")
#     # print(scot)

#     if print_scot:
#         print("=== SCoT ===")
#         print(scot)
#         print()

#     system_msg = (
#         "You are a Python coding assistant.\n"
#         "Write code based on the given solving process.\n"
#         "Note that the solving process may contain errors.\n"
#         "Output ONLY Python code.\n"
#     )

#     one_shot_code_example = """
#         def first_Repeated_Char(str):
#         \"\"\"
#         Write a python function to find the first repeated
#         character in a given string.
#         Input: str: a string
#         Output: ch: a repeated character in str
#         1: for each character ch in str:
#         2: if ch appears more than once in str:
#         3: return ch
#         4: return None
#         \"\"\"

#         h = {}
#         for ch in str:
#             if ch in h:
#                 return ch
#             else:
#                 h[ch] = 0
#         return None
#         """

#     user_msg = f"""
#         {one_shot_code_example}

#         Now generate code for the following .

#         {prompt}

#         Structured Solving Process:
#         {scot}
#         """

#     full_prompt = system_msg + "\n" + user_msg

#     raw = ollama_generate(
#         prompt=full_prompt,
#         model=model,
#         temperature=0.1,
#         num_predict=400
#     )

#     cleaned = extract_clean_code(raw)
#     return strip_function_def(prompt, cleaned)
def generate_one_completion_SCoT(prompt: str, model: str, print_scot=True) -> str:
    """
    One-shot code generation using SCoT (Figure 4 style)
    Now prints:
        1. Raw model output
        2. After extract_clean_code
        3. After strip_function_def
    """

    scot = generate_SCoT(prompt, model)

    if print_scot:
        print("=== SCoT ===")
        print(scot)
        print()

    system_msg = (
        "You are a Python coding assistant.\n"
        "Write code based on the given solving process.\n"
        "Note that the solving process may contain errors.\n"
        "Output ONLY Python code.\n"
    )

    one_shot_code_example = """
        def first_Repeated_Char(str):
        \"\"\"
        Write a python function to find the first repeated
        character in a given string.
        \"\"\"

        h = {}
        for ch in str:
            if ch in h:
                return ch
            else:
                h[ch] = 0
        return None
        """

    user_msg = f"""
        {one_shot_code_example}

        Now generate code for the following:

        {prompt}

        Structured Solving Process:
        {scot}
        """

    full_prompt = system_msg + "\n" + user_msg

    # 🔥 RAW OUTPUT
    raw = ollama_generate(
        prompt=full_prompt,
        model=model,
        temperature=0.1,
        num_predict=400
    )

    print("=== RAW MODEL OUTPUT ===")
    print(raw)
    print()

    # 🔥 AFTER extract_clean_code
    cleaned_step1 = extract_clean_code(raw)

    print("=== AFTER extract_clean_code ===")
    print(cleaned_step1)
    print()

    # 🔥 AFTER strip_function_def
    final_cleaned = strip_function_def(prompt, cleaned_step1)

    print("=== AFTER strip_function_def ===")
    print(final_cleaned)
    print()

    return final_cleaned


In [11]:
from tqdm import tqdm

# ==============================
# CONFIGURATION
# ==============================

model_name = "codellama:7b"
pass_k = 1
NUM_PROBLEMS = 5   # 🔥 Change this to control number of problems

# ==============================
# SELECT LIMITED PROBLEMS
# ==============================

problem_ids = list(problems.keys())[:NUM_PROBLEMS]

# ==============================
# GENERATION LOOP
# ==============================

for task_id in tqdm(problem_ids, desc=f"SCoT pass@{pass_k} ({model_name})"):

    problem_prompt = problems[task_id]["prompt"]

    print("\n" + "=" * 80)
    print(f"TASK ID: {task_id}")

    output = generate_one_completion_SCoT(
        problem_prompt,
        model_name,
        print_scot=True
    )

    print("=" * 80)
    print("=== FINAL RETURNED CODE ===")
    print(output)
    print("=" * 80 + "\n")

SCoT pass@1 (codellama:7b):   0%|          | 0/5 [00:00<?, ?it/s]


TASK ID: HumanEval/0
=== SCoT ===
Solving process:

1. Input: A list of numbers and a threshold value.
2. Output: Boolean value indicating if there are any two numbers in the list that are closer to each other than the given threshold.
3. Initialize a variable "close" to False.
4. Iterate over the list of numbers using a for loop.
5. For each number, calculate its distance from every other number in the list using a nested loop.
6. If any two numbers are within the specified distance of each other, set the "close" variable to True and break out of the inner loop.
7. If the inner loop completes without finding any close pairs, return False.
8. If the inner loop finds at least one close pair, return True.

Note: This solution uses a nested loop, which can be inefficient for large lists. A more efficient solution would use a single loop and keep track of the closest pair found so far instead of breaking out of the inner loop as soon as a close pair is found.



SCoT pass@1 (codellama:7b):  20%|██        | 1/5 [00:03<00:12,  3.18s/it]

=== RAW MODEL OUTPUT ===
```
def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """
    Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """
    close = False
    for i in range(len(numbers)):
        for j in range(i+1, len(numbers)):
            if abs(numbers[i] - numbers[j]) < threshold:
                close = True
                break
        if close:
            return True
    return False
```

=== AFTER extract_clean_code ===
def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """
    Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """
    close = False
 

SCoT pass@1 (codellama:7b):  40%|████      | 2/5 [00:08<00:13,  4.43s/it]

...

=== RAW MODEL OUTPUT ===
def truncate_number(number: float) -> float:
    """Given a positive floating point number, it can be decomposed into
    an integer part (largest integer smaller than given number) and decimals
    (leftover part always smaller than 1).

    Return the decimal part of the number.
    >>> truncate_number(3.5)
    0.5
    """
    integer, decimal = math.modf(number)
    while decimal != 0:
        integer += 1
        decimal = math.modf(decimal * 10)[0] / 10
    return decimal

=== AFTER extract_clean_code ===
def truncate_number(number: float) -> float:
    """Given a positive floating point number, it can be decomposed into
    an integer part (largest integer smaller than given number) and decimals
    (leftover part always smaller than 1).

    Return the decimal part of the number.
    >>> truncate_number(3.5)
    0.5
    """
    integer, decimal = math.modf(number)
    while decimal != 0:
        integer += 1
        decimal = math.modf(decimal * 10)[0] /

SCoT pass@1 (codellama:7b):  80%|████████  | 4/5 [00:13<00:03,  3.06s/it]

=== RAW MODEL OUTPUT ===
def below_zero(operations: List[int]) -> bool:
    """
    Given a list of deposit and withdrawal operations on a bank account that starts with zero balance,
    determine if the balance falls below zero at any point during the operations.

    Parameters:
        operations (List[int]): A list of deposit and withdrawal operations on a bank account.

    Returns:
        bool: True if the balance falls below zero at any point during the operations, False otherwise.
    """

    balance = 0

    for operation in operations:
        if operation > 0:
            balance += operation
        else:
            balance -= operation

        if balance < 0:
            return True

    return False

=== AFTER extract_clean_code ===
def below_zero(operations: List[int]) -> bool:
    """
    Given a list of deposit and withdrawal operations on a bank account that starts with zero balance,
    determine if the balance falls below zero at any point during the operations.

SCoT pass@1 (codellama:7b): 100%|██████████| 5/5 [00:18<00:00,  3.72s/it]

=== RAW MODEL OUTPUT ===
Here is the Python code for the `mean_absolute_deviation` function:
```python
def mean_absolute_deviation(numbers: List[float]) -> float:
    """ Calculate the Mean Absolute Deviation around the mean of a dataset.

    Args:
        numbers (List[float]): The input list of numbers.

    Returns:
        float: The Mean Absolute Deviation around the mean of this dataset.
    """
    # Calculate the mean of the input list
    mean = sum(numbers) / len(numbers)

    # Initialize a variable to store the sum of the absolute differences between each element and the mean
    total_diff = 0

    # Loop through each element in the input list
    for num in numbers:
        diff = abs(num - mean)
        total_diff += diff

    # Divide the running total by the number of elements to get the average absolute difference between each element and the mean
    mad = total_diff / len(numbers)

    return mad
```
Example usage:
```python
>>> mean_absolute_deviation([1.0, 2.0, 3

In [16]:
from tqdm import tqdm
from human_eval.data import write_jsonl

pass_k = 1  # 🔥 change this to 1, 2, 5, 10
model_name = "codellama:7b"

samples = []

for task_id in tqdm(problems, desc=f"Generating SCoT pass@{pass_k} ({model_name})"):
    prompt = problems[task_id]["prompt"]

    for _ in range(pass_k):
        samples.append({
            "task_id": task_id,
            "completion": generate_one_completion_SCoT(
                prompt,
                model_name
            )
        })

output_file = f"samples_codellama_SCoT_pass@{pass_k}.jsonl"
write_jsonl(output_file, samples)

print(f"Generated samples saved to {output_file}")


Generating SCoT pass@1 (codellama:7b): 100%|██████████| 164/164 [10:52<00:00,  3.98s/it]

Generated samples saved to samples_codellama_SCoT_pass@1.jsonl


In [1]:
! evaluate_functional_correctness samples_codellama_SCoT_pass@1.jsonl

Reading samples...
164it [00:00, 8979.97it/s]
Running test suites...
100%|█████████████████████████████████████████| 164/164 [00:05<00:00, 29.39it/s]
Writing results to samples_codellama_SCoT_pass@1.jsonl_results.jsonl...
100%|██████████████████████████████████████| 164/164 [00:00<00:00, 30429.81it/s]
{'pass@1': 0.11585365853658537}


In [37]:
from tqdm import tqdm
from human_eval.data import write_jsonl

pass_k = 1  # 🔥 change this to 1, 2, 5, 10
model_name = "codeqwen:7b"

samples = []

for task_id in tqdm(problems, desc=f"Generating SCoT pass@{pass_k} ({model_name})"):
    prompt = problems[task_id]["prompt"]

    for _ in range(pass_k):
        samples.append({
            "task_id": task_id,
            "completion": generate_one_completion_SCoT(
                prompt,
                model_name
            )
        })

output_file = f"samples_codeqwen_SCoT_pass@{pass_k}.jsonl"
write_jsonl(output_file, samples)

print(f"Generated samples saved to {output_file}")


Generating SCoT pass@1 (codeqwen:7b): 100%|██████████| 164/164 [16:03<00:00,  5.87s/it]

Generated samples saved to samples_codeqwen_SCoT_pass@1.jsonl


In [ ]:
! evaluate_functional_correctness samples_codeqwen_SCoT_pass@1.jsonl

10644.45s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Reading samples...
164it [00:00, 19199.11it/s]
Running test suites...
100%|█████████████████████████████████████████| 164/164 [00:01<00:00, 89.00it/s]
Writing results to samples_codeqwen_SCoT_pass@1.jsonl_results.jsonl...
100%|██████████████████████████████████████| 164/164 [00:00<00:00, 32101.26it/s]
{'pass@1': 0.42073170731707316}


In [26]:
from tqdm import tqdm
from human_eval.data import write_jsonl

pass_k = 1  # pass@k (1, 2, 5, 10)
model_name = "codellama:7b"

samples = []

for task_id in tqdm(
    problems,
    desc=f"Generating ONE-SHOT SCoT pass@{pass_k} ({model_name})"
):
    prompt = problems[task_id]["prompt"]

    for _ in range(pass_k):
        samples.append({
            "task_id": task_id,
            "completion": generate_one_completion_SCoT(
                prompt,
                model_name
            )
        })

# ✅ Explicitly mark ONE-SHOT in filename
output_file = f"samples_codellama_ONE_SHOT_SCoT_pass@{pass_k}.jsonl"
write_jsonl(output_file, samples)

print(f"Generated samples saved to {output_file}")


Generating ONE-SHOT SCoT pass@1 (codellama:7b): 100%|██████████| 164/164 [11:45<00:00,  4.30s/it]

Generated samples saved to samples_codellama_ONE_SHOT_SCoT_pass@1.jsonl


In [27]:
! evaluate_functional_correctness samples_codellama_ONE_SHOT_SCoT_pass@1.jsonl

Reading samples...
164it [00:00, 8813.26it/s]
Running test suites...
100%|█████████████████████████████████████████| 164/164 [00:04<00:00, 35.44it/s]
Writing results to samples_codellama_ONE_SHOT_SCoT_pass@1.jsonl_results.jsonl...
100%|██████████████████████████████████████| 164/164 [00:00<00:00, 25448.24it/s]
{'pass@1': 0.1951219512195122}


In [28]:
from tqdm import tqdm
from human_eval.data import write_jsonl

pass_k = 1  # pass@k (1, 2, 5, 10)
model_name = "codeqwen:7b"

samples = []

for task_id in tqdm(
    problems,
    desc=f"Generating ONE-SHOT SCoT pass@{pass_k} ({model_name})"
):
    prompt = problems[task_id]["prompt"]

    for _ in range(pass_k):
        samples.append({
            "task_id": task_id,
            "completion": generate_one_completion_SCoT(
                prompt,
                model_name
            )
        })

# ✅ Explicitly mark ONE-SHOT in filename
output_file = f"samples_codeqwen_ONE_SHOT_SCoT_pass@{pass_k}.jsonl"
write_jsonl(output_file, samples)

print(f"Generated samples saved to {output_file}")


Generating ONE-SHOT SCoT pass@1 (codeqwen:7b): 100%|██████████| 164/164 [11:43<00:00,  4.29s/it]

Generated samples saved to samples_codeqwen_ONE_SHOT_SCoT_pass@1.jsonl


In [29]:
! evaluate_functional_correctness samples_codeqwen_ONE_SHOT_SCoT_pass@1.jsonl

Reading samples...
164it [00:00, 12490.53it/s]
Running test suites...
100%|█████████████████████████████████████████| 164/164 [00:05<00:00, 28.45it/s]
Writing results to samples_codeqwen_ONE_SHOT_SCoT_pass@1.jsonl_results.jsonl...
100%|██████████████████████████████████████| 164/164 [00:00<00:00, 26870.81it/s]
{'pass@1': 0.40853658536585363}


In [ ]:
# ============================================================
# Paper-faithful SCoT Prompting + Code Generation
# WITH detailed progress logging (ONE CELL)
# ============================================================

import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from human_eval.data import write_jsonl

OLLAMA_URL = "http://127.0.0.1:11434/api/generate"
MODEL_NAME = "codellama:7b"
N = 20  # number of samples per problem (paper setting)

# ---------- Ollama wrapper ----------
def ollama_generate(prompt, model, temperature, num_predict):
    payload = {
        "model": model,
        "prompt": prompt,
        "temperature": temperature,
        "num_predict": num_predict,
        "stream": False
    }
    r = requests.post(OLLAMA_URL, json=payload, timeout=None)
    r.raise_for_status()
    return r.json()["response"].strip()


# ---------- Generate 20 independent SCoTs (Figure 3) ----------
def generate_20_scots(prompt, model, prob_idx, total_probs):
    system_msg = (
        "You are a Python problem-solving assistant.\n"
        "Generate a rough structured solving process in natural language.\n"
        "Do NOT write code.\n"
    )

    three_shot_example = """

                        Example 1:-
                        def first_Repeated_Char(str):
                        \"\"\"
                        Write a python function to find the first repeated
                        character in a given string.
                        \"\"\"
                        Pass


                        Please understand the requirement and write a rough solving
                        process. It starts with a input-output structure. You
                        should use three basic structures to build the solving
                        process, including sequences, branches, and loops.
                        The necessary details should be written in natural languages.


                        Input: str: a string
                        Output: ch: a repeated character in str
                        1: for each character ch in str:
                        2: if ch appears more than once in str:
                        3: return ch
                        4: return None


                        Example 2:-
                        def sum_of_squares(nums):
                        \"\"\"
                        Write a python function to compute the sum of squares of numbers.
                        \"\"\"
                        Pass


                        Please understand the requirement and write a rough solving
                        process. It starts with a input-output structure. You
                        should use three basic structures to build the solving
                        process, including sequences, branches, and loops.
                        The necessary details should be written in natural languages.


                        Input: nums: list[int]
                        Output: total: int
                        1: initialize total to 0
                        2: for each number n in nums:
                        3: add n * n to total
                        4: return total


                        Example 3:-
                        def is_palindrome(s):
                        \"\"\"
                        Write a python function to check if a string is a palindrome.
                        \"\"\"
                        Pass


                        Please understand the requirement and write a rough solving
                        process. It starts with a input-output structure. You
                        should use three basic structures to build the solving
                        process, including sequences, branches, and loops.
                        The necessary details should be written in natural languages.


                        Input: s: str
                        Output: result: bool
                        1: reverse the string s
                        2: if reversed string equals s:
                        3: return True
                        4: else return False
                        """

    full_prompt = (
        system_msg
        + "\n"
        + three_shot_example
        + f"\n\nNow solve the following problem in the SAME way.\n\n{prompt}"
    )

    scots = []
    with ThreadPoolExecutor(max_workers=N) as ex:
        futures = {
            ex.submit(ollama_generate, full_prompt, model, 0.8, 400): i
            for i in range(1, N + 1)
        }
        for f in as_completed(futures):
            idx = futures[f]
            scots.append(f.result())
            print(f"[Problem {prob_idx}/{total_probs}] SCoT {idx} generated")

    return scots

# ---------- Generate 1 code per SCoT (Figure 4) ----------
def generate_codes_from_scots(prompt, scots, model, prob_idx, total_probs):
    system_msg = (
        "You are a Python coding assistant.\n"
        "Write code based on the given solving process.\n"
        "Note that the solving process may contain errors.\n"
        "Output ONLY Python code.\n"
    )

    three_shot_code_example = """
                        Example 1:-
                            def first_Repeated_Char(str):
                            \"\"\"
                            Write a python function to find the first repeated character
                            in a given string.
                            Input: str: a string
                            Output: ch: a repeated character in str
                            1: for each character ch in str:
                            2: if ch appears more than once in str:
                            3: return ch
                            4: return None
                            \"\"\"


                            h = {}
                            for ch in str:
                                if ch in h:
                                    return ch
                                else:
                                    h[ch] = 0
                            return None


                        Example 2:-
                            def sum_of_squares(nums):
                            \"\"\"
                            Input: nums: list[int]
                            Output: total: int
                            1: initialize total to 0
                            2: for each number n in nums:
                            3: add n * n to total
                            4: return total
                            \"\"\"


                            total = 0
                            for n in nums:
                                total += n * n
                            return total


                        Example 3:-
                            def is_palindrome(s):
                            \"\"\"
                            Input: s: str
                            Output: result: bool
                            1: reverse the string s
                            2: if reversed string equals s:
                            3: return True
                            4: else return False
                            \"\"\"


                            return s == s[::-1]
                            """


    def build_prompt(scot):
        return (
            system_msg
            + "\n"
            + three_shot_code_example
            + f"\n\nProblem:\n{prompt}\n\nStructured Solving Process:\n{scot}"
        )

    codes = []
    with ThreadPoolExecutor(max_workers=len(scots)) as ex:
        futures = {
            ex.submit(ollama_generate, build_prompt(scot), model, 0.1, 300): i
            for i, scot in enumerate(scots, start=1)
        }
        for f in as_completed(futures):
            idx = futures[f]
            codes.append(f.result())
            print(f"[Problem {prob_idx}/{total_probs}] Code {idx} generated")

    return codes


# ---------- Main loop ----------
samples = []
total_probs = len(problems)

for prob_idx, task_id in enumerate(problems, start=1):
    prompt = problems[task_id]["prompt"]

    print(f"\n=== Starting Problem {prob_idx}/{total_probs} ===")

    scots = generate_20_scots(prompt, MODEL_NAME, prob_idx, total_probs)
    codes = generate_codes_from_scots(prompt, scots, MODEL_NAME, prob_idx, total_probs)

    # IMPORTANT: store ALL 20 codes (no selection!)
    for code in codes:
        samples.append({
            "task_id": task_id,
            "completion": code
        })

output_file = f"samples_{MODEL_NAME.replace(':','_')}_SCoT_n{N}.jsonl"
write_jsonl(output_file, samples)

print(f"\n✅ Saved {len(samples)} samples to {output_file}")


=== Starting Problem 1/164 ===
[Problem 1/164] SCoT 1 generated
[Problem 1/164] SCoT 6 generated
[Problem 1/164] SCoT 2 generated
[Problem 1/164] SCoT 3 generated
[Problem 1/164] SCoT 4 generated
[Problem 1/164] SCoT 5 generated
[Problem 1/164] SCoT 12 generated
[Problem 1/164] SCoT 9 generated
[Problem 1/164] SCoT 13 generated
[Problem 1/164] SCoT 7 generated
[Problem 1/164] SCoT 16 generated
[Problem 1/164] SCoT 11 generated
[Problem 1/164] SCoT 10 generated
[Problem 1/164] SCoT 8 generated
[Problem 1/164] SCoT 17 generated
[Problem 1/164] SCoT 14 generated
[Problem 1/164] SCoT 18 generated
[Problem 1/164] SCoT 20 generated
[Problem 1/164] SCoT 19 generated
[Problem 1/164] SCoT 15 generated
[Problem 1/164] Code 6 generated
[Problem 1/164] Code 2 generated
[Problem 1/164] Code 3 generated
[Problem 1/164] Code 5 generated
[Problem 1/164] Code 4 generated
[Problem 1/164] Code 1 generated
[Problem 1/164] Code 7 generated
[Problem 1/164] Code 11 generated
[Problem 1/164] Code 12 generate

In [12]:
! evaluate_functional_correctness samples_codellama_7b_SCoT_n20.jsonl

Reading samples...
3280it [00:00, 13231.43it/s]
Running test suites...
100%|███████████████████████████████████████| 3280/3280 [01:02<00:00, 52.52it/s]
Writing results to samples_codellama_7b_SCoT_n20.jsonl_results.jsonl...
100%|████████████████████████████████████| 3280/3280 [00:00<00:00, 34317.37it/s]
{'pass@1': 0.03353658536585367, 'pass@10': 0.2542003322510005, 'pass@20': 0.40853658536585363}


# SCOT + PHP

In [11]:
import requests

OLLAMA_URL = "http://127.0.0.1:11434/api/generate"

def ollama_generate(prompt: str, model: str, temperature=0.1, num_predict=500) -> str:
    payload = {
        "model": model,
        "prompt": prompt,
        "temperature": temperature,
        "num_predict": num_predict,
        "stream": False
    }

    resp = requests.post(OLLAMA_URL, json=payload, timeout=300)
    resp.raise_for_status()

    return resp.json().get("response", "").strip()


In [12]:
def generate_SCoT_PHP(
    prompt: str,
    model: str,
    max_rounds: int = 3,
    temperature: float = 0.8
) -> str:
    """
    Progressive-Hint Prompting for SCoT generation
    Uses previous SCoTs as hints until stabilization
    """

    system_msg = (
        "You are a Python problem-solving assistant.\n"
        "Generate a rough structured solving process in natural language.\n"
        "Do NOT write code.\n"
    )

    one_shot_example = """
            def first_Repeated_Char(str):
            \"\"\"
            Write a python function to find the first repeated
            character in a given string.
            \"\"\"
            Pass

            Input: str: a string
            Output: ch: a repeated character in str
            1: for each character ch in str:
            2: if ch appears more than once in str:
            3: return ch
            4: return None
            """

    previous_scot = None
    current_scot = None

    for round_id in range(max_rounds):
        if previous_scot is None:
            # Round 1: normal SCoT
            user_msg = f"""
                        {one_shot_example}

                        Now solve the following problem in the SAME way.

                        {prompt}
                        """
        else:
            # 🔥 PHP hint injection
            user_msg = f"""
                        Previously, you gave the following solving process:

                        {previous_scot}

                        Please re-think the problem and generate a refined structured solving process.

                        Problem:
                        {prompt}
                        """

        full_prompt = system_msg + "\n" + user_msg

        current_scot = ollama_generate(
            prompt=full_prompt,
            model=model,
            temperature=temperature,
            num_predict=500
        )

        # PHP stopping condition (stability)
        if previous_scot is not None and current_scot.strip() == previous_scot.strip():
            break

        previous_scot = current_scot

    return current_scot


In [ ]:
def generate_one_completion_SCoT_PHP(
    prompt: str,
    model: str,
    print_scot: bool = False
) -> str:
    """
    Code generation using SCoT + Progressive-Hint Prompting
    """

    # 🔥 CHANGED: PHP-based SCoT
    scot = generate_SCoT_PHP(prompt, model)

    if print_scot:
        print("=== FINAL STABLE SCoT (PHP) ===")
        print(scot)
        print()

    system_msg = (
        "You are a Python coding assistant.\n"
        "Write code based on the given solving process.\n"
        "Note that the solving process may contain errors.\n"
        "Output ONLY Python code.\n"
    )

    one_shot_code_example = """
                            def first_Repeated_Char(str):
                            \"\"\"
                            Write a python function to find the first repeated
                            character in a given string.
                            Input: str: a string
                            Output: ch: a repeated character in str
                            1: for each character ch in str:
                            2: if ch appears more than once in str:
                            3: return ch
                            4: return None
                            \"\"\"

                            h = {}
                            for ch in str:
                                if ch in h:
                                    return ch
                                else:
                                    h[ch] = 0
                            return None
                            """

    user_msg = f"""
                {one_shot_code_example}

                Now generate code for the following problem.

                Problem:
                {prompt}

                Structured Solving Process:
                {scot}
                """

    full_prompt = system_msg + "\n" + user_msg

    raw = ollama_generate(
        prompt=full_prompt,
        model=model,
        temperature=0.1,  # low temp for code
        num_predict=400
    )

    cleaned = extract_clean_code(raw)
    return strip_function_def(prompt, cleaned)


In [ ]:
from tqdm import tqdm
from human_eval.data import write_jsonl

pass_k = 1
model_name = "codellama:7b"

samples = []

for task_id in tqdm(
    problems,
    desc=f"Generating PHP + SCoT pass@{pass_k} ({model_name})"
):
    prompt = problems[task_id]["prompt"]

    for _ in range(pass_k):
        samples.append({
            "task_id": task_id,
            "completion": generate_one_completion_SCoT_PHP(
                prompt,
                model_name
            )
        })

output_file = f"samples_codellama_PHP_SCoT_pass@{pass_k}.jsonl"
write_jsonl(output_file, samples)

print(f"Generated samples saved to {output_file}")


Generating PHP + SCoT pass@1 (codellama:7b):  34%|███▍      | 56/164 [36:00<1:06:01, 36.69s/it]

# SELF PLANNING

In [12]:
import requests

OLLAMA_URL = "http://127.0.0.1:11434/api/generate"

def ollama_generate(prompt: str, model: str, temperature=0.1, num_predict=500) -> str:
    payload = {
        "model": model,
        "prompt": prompt,
        "temperature": temperature,
        "num_predict": num_predict,
        "stream": False
    }

    resp = requests.post(OLLAMA_URL, json=payload, timeout=300)
    resp.raise_for_status()
    return resp.json().get("response", "").strip()


In [13]:
def generate_plan(prompt: str, model: str) -> str:
    """
    Generates a high-level self-planning step-by-step plan (NO CODE).
    """

    system_msg = (
        "You are a Python problem-solving assistant.\n"
        "First, generate a high-level step-by-step plan.\n"
        "Do NOT write any code.\n"
    )

    user_msg = f"""
    Please create a structured plan for solving the following coding problem.

    Rules:
    1. Use a numbered list.
    2. Start each step with an action verb (Initialize, Check, Loop, Return).
    3. Include conditionals and loops where needed.
    4. Stay high-level (no syntax, no code).
    5. Keep the steps at a high level; avoid low-level implementation details.

    Problem:
    {prompt}
    """

    full_prompt = system_msg + "\n" + user_msg

    try:
        return ollama_generate(
            prompt=full_prompt,
            model=model,
            temperature=0.1,     # mild diversity in planning
            num_predict=400
        )
    except Exception as e:
        print("[Self-Planning error]", e)
        return ""


In [ ]:
def generate_one_completion_SelfPlanning(prompt: str, model: str, print_plan=False) -> str:
    """
    Generates code using Self-Planning + local CodeLLaMA.
    """

    try:
        # Step 1: Generate plan
        plan = generate_plan(prompt, model)

        if print_plan:
            print("=== SELF PLAN ===")
            print(plan)
            print()

        system_msg = (
            "You are a Python coding assistant.\n"
            "Use the provided plan to implement the function.\n"
            "Rules:\n"
            "- Output ONLY Python code\n"
            "- Do NOT repeat the function signature\n"
            "- Do NOT add comments or explanations\n"
        )

        user_msg = f"""
            Problem:
            {prompt}

            Structured Plan:
            {plan}
            """

        full_prompt = system_msg + "\n" + user_msg

        raw = ollama_generate(
            prompt=full_prompt,
            model=model,
            temperature=0.1,     # deterministic code gen
            num_predict=256
        )

        cleaned = extract_clean_code(raw)
        return strip_function_def(prompt, cleaned)

    except Exception as e:
        print("[Self-Planning code error]", e)
        return "    pass"


In [18]:
from tqdm import tqdm
from human_eval.data import write_jsonl

pass_k = 1            # 🔥 change to 1, 2, 5, 10
model_name = "codellama:7b"

samples = []

for task_id in tqdm(problems, desc=f"Generating Self-Planning pass@{pass_k} ({model_name})"):
    prompt = problems[task_id]["prompt"]

    for _ in range(pass_k):
        samples.append({
            "task_id": task_id,
            "completion": generate_one_completion_SelfPlanning(
                prompt,
                model_name
            )
        })

output_file = f"samples_codellama_SelfPlanning_pass@{pass_k}.jsonl"
write_jsonl(output_file, samples)

print(f"Generated samples saved to {output_file}")


Generating Self-Planning pass@1 (codellama:7b): 100%|██████████| 164/164 [05:58<00:00,  2.18s/it]

Generated samples saved to samples_codellama_SelfPlanning_pass@1.jsonl


In [19]:
! evaluate_functional_correctness samples_codellama_SelfPlanning_pass@1.jsonl

Reading samples...
164it [00:00, 11819.81it/s]
Running test suites...
100%|█████████████████████████████████████████| 164/164 [00:03<00:00, 53.98it/s]
Writing results to samples_codellama_SelfPlanning_pass@1.jsonl_results.jsonl...
100%|██████████████████████████████████████| 164/164 [00:00<00:00, 31728.13it/s]
{'pass@1': 0.2682926829268293}


In [21]:
from tqdm import tqdm
from human_eval.data import write_jsonl

pass_k = 1            # 🔥 change to 1, 2, 5, 10
model_name = "codeqwen:7b"

samples = []

for task_id in tqdm(problems, desc=f"Generating Self-Planning pass@{pass_k} ({model_name})"):
    prompt = problems[task_id]["prompt"]

    for _ in range(pass_k):
        samples.append({
            "task_id": task_id,
            "completion": generate_one_completion_SelfPlanning(
                prompt,
                model_name
            )
        })

output_file = f"samples_codeqwen_SelfPlanning_pass@{pass_k}.jsonl"
write_jsonl(output_file, samples)

print(f"Generated samples saved to {output_file}")


Generating Self-Planning pass@1 (codeqwen:7b): 100%|██████████| 164/164 [07:29<00:00,  2.74s/it]

Generated samples saved to samples_codeqwen_SelfPlanning_pass@1.jsonl


In [23]:
! evaluate_functional_correctness samples_codeqwen_SelfPlanning_pass@1.jsonl

Reading samples...
164it [00:00, 9858.91it/s]
Running test suites...
100%|█████████████████████████████████████████| 164/164 [00:04<00:00, 40.72it/s]
Writing results to samples_codeqwen_SelfPlanning_pass@1.jsonl_results.jsonl...
100%|██████████████████████████████████████| 164/164 [00:00<00:00, 29918.92it/s]
{'pass@1': 0.4817073170731707}


# Self-Consistency

In [10]:
import ast
from collections import deque
from typing import Set, List

# --- Step 1: Per-function canonicalization ---
class CanonicalizeFunction(ast.NodeTransformer):
    def visit_FunctionDef(self, node: ast.FunctionDef):
        # --- a) Canonicalize the function name ---
        # Replace the function name with a fixed token (e.g., "func")
        node.name = "func"
        
        # --- b) Build a canonical mapping for identifiers ---
        # First, preserve parameters in order.
        param_names = [arg.arg for arg in node.args.args]
        mapping = {}
        for i, name in enumerate(param_names):
            mapping[name] = f"var_{i}"
        
        # Collect all other names from the function body.
        other_names = set()
        class NameCollector(ast.NodeVisitor):
            def visit_Name(self, n):
                if n.id not in mapping:
                    other_names.add(n.id)
            def visit_arg(self, n):
                # Do nothing here since parameters are handled.
                pass
        NameCollector().visit(node)
        
        # Sort the remaining names and assign them canonical names.
        for name in sorted(other_names):
            mapping[name] = f"var_{len(mapping)}"
        
        # --- c) Replace all Name and arg nodes using the mapping ---
        class NameReplacer(ast.NodeTransformer):
            def visit_Name(self, n):
                if n.id in mapping:
                    n.id = mapping[n.id]
                return n
            def visit_arg(self, n):
                if n.arg in mapping:
                    n.arg = mapping[n.arg]
                return n
        node = NameReplacer().visit(node)
        ast.fix_missing_locations(node)
        return node

class GlobalCanonicalizer(ast.NodeTransformer):
    def visit_Module(self, node: ast.Module):
        new_body = []
        for stmt in node.body:
            if isinstance(stmt, ast.FunctionDef):
                new_body.append(CanonicalizeFunction().visit(stmt))
            else:
                new_body.append(stmt)
        node.body = new_body
        return node

# --- Step 2: Reordering of Simple, Independent Assignments ---
def reorder_assignments_in_block(block: List[ast.stmt]) -> List[ast.stmt]:
    indices = []
    nodes = []
    for i, stmt in enumerate(block):
        if isinstance(stmt, ast.Assign) and len(stmt.targets) == 1 and isinstance(stmt.targets[0], ast.Name):
            indices.append(i)
            nodes.append(stmt)
    if not nodes:
        return block

    assigned_names = [node.targets[0].id for node in nodes]
    deps = []
    for node in nodes:
        free_vars = set()
        for child in ast.walk(node.value):
            if isinstance(child, ast.Name):
                free_vars.add(child.id)
        dep = {name for name in free_vars if name in assigned_names}
        deps.append(dep)
    
    # For independent assignments (with no dependencies on the other assignments in this block),
    # sort them by a canonical key.
    indep_idxs = [i for i, d in enumerate(deps) if not d]
    sorted_indep = sorted([nodes[i] for i in indep_idxs],
                          key=lambda n: ast.dump(n, annotate_fields=True, include_attributes=False))
    
    new_nodes = list(nodes)
    for order, i in enumerate(indep_idxs):
        new_nodes[i] = sorted_indep[order]
    
    new_block = list(block)
    for pos, original_idx in enumerate(indices):
        new_block[original_idx] = new_nodes[pos]
    return new_block

class ReorderAssignments(ast.NodeTransformer):
    def generic_visit(self, node):
        if hasattr(node, 'body') and isinstance(node.body, list):
            new_body = []
            current_block = []
            for stmt in node.body:
                if isinstance(stmt, ast.Assign) and len(stmt.targets) == 1 and isinstance(stmt.targets[0], ast.Name):
                    current_block.append(stmt)
                else:
                    if current_block:
                        new_body.extend(reorder_assignments_in_block(current_block))
                        current_block = []
                    new_body.append(stmt)
            if current_block:
                new_body.extend(reorder_assignments_in_block(current_block))
            node.body = new_body
        return super().generic_visit(node)

# --- Step 3: Normalize simple return tuples ---
class NormalizeReturnTuples(ast.NodeTransformer):
    def visit_Return(self, node: ast.Return):
        node = self.generic_visit(node)
        if isinstance(node.value, ast.Tuple):
            if all(isinstance(elt, (ast.Name, ast.Constant)) for elt in node.value.elts):
                sorted_elts = sorted(node.value.elts,
                                     key=lambda n: ast.dump(n, annotate_fields=True, include_attributes=False))
                node.value.elts = sorted_elts
        return node

# --- Step 4: Combine All Transformations ---
class PerfectNormalizer(ast.NodeTransformer):
    def __init__(self):
        self.global_canonicalizer = GlobalCanonicalizer()
        self.reorder_assignments = ReorderAssignments()
        self.normalize_return_tuples = NormalizeReturnTuples()
    
    def visit(self, node):
        # Global canonicalization (function-level)
        node = self.global_canonicalizer.visit(node)
        ast.fix_missing_locations(node)
        # Reorder contiguous simple assignments
        node = self.reorder_assignments.visit(node)
        ast.fix_missing_locations(node)
        # Normalize simple return tuples
        node = self.normalize_return_tuples.visit(node)
        ast.fix_missing_locations(node)
        return node

def perfect_normalize_code(source: str) -> str:
    try:
        tree = ast.parse(source)
        normalized_tree = PerfectNormalizer().visit(tree)
        ast.fix_missing_locations(normalized_tree)
        return ast.dump(normalized_tree, annotate_fields=True, include_attributes=False)
    except Exception as e:
        print("Error in perfect_normalize_code:", e)
        return ""

# --- Test Suite and Runner ---
test_cases = [
    {
        "description": "Test 1: Basic add function with different variable names and order of assignments",
        "code1": """
def add(a, b):
    x = a + b
    y = a - b
    return x, y
""",
        "code2": """
def add(x, y):
    y_temp = x - y
    x_temp = x + y
    return x_temp, y_temp
""",
        "expected": True
    },
    {
        "description": "Test 2: Multiply and add with different naming and order",
        "code1": """
def multiply_and_add(a, b, c):
    result1 = a * b
    result2 = a + c
    return result1, result2
""",
        "code2": """
def multiply_and_add(x, y, z):
    temp = x + z
    prod = x * y
    return prod, temp
""",
        "expected": True
    },
    {
        "description": "Test 3: Independent assignments reordered in function",
        "code1": """
def compute():
    a = 5
    b = 10
    c = a + b
    return a, b, c
""",
        "code2": """
def compute():
    b = 10
    a = 5
    c = a + b
    return a, b, c
""",
        "expected": True
    },
    {
        "description": "Test 4: Nested tuple differences (should be non-equivalent)",
        "code1": """
def differences(a, b):
    diff = a - b
    sum_val = a + b
    return diff, sum_val, (a, b)
""",
        "code2": """
def differences(x, y):
    diff = x - y
    sum_val = x + y
    return diff, sum_val, (y, x)
""",
        "expected": False
    },
    {
        "description": "Test 5: Comments and whitespace differences",
        "code1": """
def greet(name):
    # This function greets a person.
    return "Hello, " + name
""",
        "code2": """
def greet(n):
    return "Hello, " + n  # greet the user
""",
        "expected": True
    },
    {
        "description": "Test 6: Non contiguous assignment block with a print statement",
        "code1": """
def process(x):
    a = x + 1
    b = x * 2
    print(a)
    c = a + b
    return c
""",
        "code2": """
def process(y):
    b = y * 2
    a = y + 1
    print(a)
    c = a + b
    return c
""",
        "expected": True
    },
    {
        "description": "Test 7: Control flow differences with if/else (not normalized)",
        "code1": """
def check(n):
    if n > 0:
        result = "positive"
    else:
        result = "non-positive"
    return result
""",
        "code2": """
def check(x):
    result = "non-positive"
    if x > 0:
        result = "positive"
    return result
""",
        "expected": False
    },
    {
        "description": "Test 8: List comprehension vs. for-loop (structure differs)",
        "code1": """
def squares(nums):
    return [x*x for x in nums]
""",
        "code2": """
def squares(nums):
    result = []
    for n in nums:
        result.append(n*n)
    return result
""",
        "expected": False
    },
    {
        "description": "Test 9: Extra parentheses differences",
        "code1": """
def subtract(a, b):
    result = (a - b)
    return result
""",
        "code2": """
def subtract(x, y):
    result = x - y
    return result
""",
        "expected": True
    },
    {
        "description": "Test 10: Multiple assignments and reordered return tuple",
        "code1": """
def stats(data):
    total = sum(data)
    count = len(data)
    avg = total / count
    min_val = min(data)
    max_val = max(data)
    return avg, min_val, max_val
""",
        "code2": """
def stats(values):
    count = len(values)
    total = sum(values)
    avg = total / count
    max_val = max(values)
    min_val = min(values)
    return avg, max_val, min_val
""",
        "expected": True
    }
]

def run_tests():
    passed = 0
    for i, test in enumerate(test_cases, 1):
        norm1 = perfect_normalize_code(test["code1"])
        norm2 = perfect_normalize_code(test["code2"])
        equivalent = (norm1 == norm2)
        result = "PASSED" if equivalent == test["expected"] else "FAILED"
        print(f"{test['description']}")
        print(f"Expected: {test['expected']} | Got: {equivalent} => {result}\n")
        if result == "PASSED":
            passed += 1
    print(f"Final Score: {passed} / {len(test_cases)} tests passed.")

if __name__ == "__main__":
    run_tests()


Test 1: Basic add function with different variable names and order of assignments
Expected: True | Got: True => PASSED

Test 2: Multiply and add with different naming and order
Expected: True | Got: True => PASSED

Test 3: Independent assignments reordered in function
Expected: True | Got: True => PASSED

Test 4: Nested tuple differences (should be non-equivalent)
Expected: False | Got: False => PASSED

Test 5: Comments and whitespace differences
Expected: True | Got: True => PASSED

Test 6: Non contiguous assignment block with a print statement
Expected: True | Got: True => PASSED

Test 7: Control flow differences with if/else (not normalized)
Expected: False | Got: False => PASSED

Test 8: List comprehension vs. for-loop (structure differs)
Expected: False | Got: False => PASSED

Test 9: Extra parentheses differences
Expected: True | Got: True => PASSED

Test 10: Multiple assignments and reordered return tuple
Expected: True | Got: True => PASSED

Final Score: 10 / 10 tests passed.


In [29]:
import requests

OLLAMA_URL = "http://127.0.0.1:11434/api/generate"

def ollama_generate(
    prompt: str,
    model: str,
    temperature: float,
    max_tokens: int
) -> str:
    """
    Calls local Ollama via HTTP API and returns raw text output.
    """

    payload = {
        "model": model,
        "prompt": prompt,
        "temperature": temperature,
        "num_predict": max_tokens,
        "stream": False
    }

    resp = requests.post(OLLAMA_URL, json=payload, timeout=300)
    resp.raise_for_status()

    return resp.json().get("response", "").strip()


In [32]:
from collections import Counter

def generate_one_completion_self_consistency(
    prompt: str,
    model: str,
    num_samples: int = 5,      # original value
    temperature: float = 0.8,
    max_tokens: int = 400
) -> str:
    """
    Self-Consistency (faithful to original implementation).

    Generates multiple completions, normalizes them,
    and returns the most consistent (majority-voted) code.
    """

    system_prompt = (
        "You are a Python coding assistant. "
        "Generate correct and complete Python code only."
        "do not add any comments"
    )

    completions = []

    for _ in range(num_samples):
        try:
            raw = ollama_generate(
                prompt=(
                    f"{system_prompt}\n\n"
                    f"Complete the following function: (do not add any comments) \n\n{prompt}"
                ),
                model=model,
                temperature=temperature,
                max_tokens=max_tokens
            )

            cleaned = extract_clean_code(raw)
            completions.append(cleaned)

        except Exception:
            continue

    normalized_to_original = {}
    normalized_forms = []

    for code in completions:
        try:
            norm = perfect_normalize_code(code)
            normalized_forms.append(norm)
            normalized_to_original.setdefault(norm, []).append(code)
        except Exception:
            continue

    if not normalized_forms:
        return "def solution():\n    pass"

    counter = Counter(normalized_forms)
    most_common_norm, _ = counter.most_common(1)[0]

    most_common_code = normalized_to_original[most_common_norm][0]
    return most_common_code


In [ ]:
from tqdm import tqdm

model_name = "codellama:7b"   # local Ollama model

samples = []

for task_id in tqdm(problems, desc="Generating Self-Consistency completions"):
    best_code = generate_one_completion_self_consistency(
        prompt=problems[task_id]["prompt"],
        model=model_name
        # num_samples, temperature, max_tokens
        # are already defined INSIDE the function
    )

    samples.append({
        "task_id": task_id,
        "completion": best_code
    })

# Explicit filename showing pass@1
output_file = "samples_ollama_SelfConsistency_pass@1.jsonl"

write_jsonl(output_file, samples)

print(f"Generated samples saved to {output_file}")


Generating Self-Consistency completions:   1%|          | 1/164 [00:05<14:35,  5.37s/it]

Error in perfect_normalize_code: invalid syntax (<unknown>, line 7)


Generating Self-Consistency completions:   4%|▎         | 6/164 [00:18<07:25,  2.82s/it]

Error in perfect_normalize_code: invalid syntax (<unknown>, line 3)
Error in perfect_normalize_code: invalid syntax (<unknown>, line 3)


Generating Self-Consistency completions:   5%|▍         | 8/164 [00:23<06:46,  2.61s/it]

...

Error in perfect_normalize_code: invalid syntax (<unknown>, line 5)


Generating Self-Consistency completions:  79%|███████▉  | 130/164 [07:51<05:03,  8.94s/it]

Error in perfect_normalize_code: invalid syntax (<unknown>, line 17)


Generating Self-Consistency completions: 100%|██████████| 164/164 [09:42<00:00,  3.55s/it]

Generated samples saved to samples_codellama_SelfConsistency_pass@1.jsonl


In [34]:
! evaluate_functional_correctness samples_ollama_SelfConsistency_pass@1.jsonl

Reading samples...
164it [00:00, 22146.36it/s]
Running test suites...
100%|█████████████████████████████████████████| 164/164 [00:04<00:00, 38.81it/s]
Writing results to samples_ollama_SelfConsistency_pass@1.jsonl_results.jsonl...
100%|██████████████████████████████████████| 164/164 [00:00<00:00, 35709.18it/s]
{'pass@1': 0.4146341463414634}


In [37]:
from tqdm import tqdm

model_name = "codeqwen:7b"   # local Ollama model

samples = []

for task_id in tqdm(problems, desc="Generating Self-Consistency completions"):
    best_code = generate_one_completion_self_consistency(
        prompt=problems[task_id]["prompt"],
        model=model_name
        # num_samples, temperature, max_tokens
        # are already defined INSIDE the function
    )

    samples.append({
        "task_id": task_id,
        "completion": best_code
    })

# Explicit filename showing pass@1
output_file = "samples_codeqwen_SelfConsistency_pass@1.jsonl"

write_jsonl(output_file, samples)

print(f"Generated samples saved to {output_file}")


Generating Self-Consistency completions:  49%|████▉     | 80/164 [09:14<08:21,  5.97s/it]

Error in perfect_normalize_code: EOF while scanning triple-quoted string literal (<unknown>, line 10)


Generating Self-Consistency completions:  68%|██████▊   | 111/164 [13:26<08:03,  9.12s/it]

Error in perfect_normalize_code: invalid syntax (<unknown>, line 3)


Generating Self-Consistency completions:  84%|████████▍ | 138/164 [17:19<03:34,  8.24s/it]

...

Error in perfect_normalize_code: unexpected EOF while parsing (<unknown>, line 2)


Generating Self-Consistency completions:  94%|█████████▍| 154/164 [19:31<01:12,  7.20s/it]

Error in perfect_normalize_code: invalid syntax (<unknown>, line 5)


Generating Self-Consistency completions: 100%|██████████| 164/164 [20:45<00:00,  7.59s/it]

Generated samples saved to samples_codeqwen_SelfConsistency_pass@1.jsonl


In [38]:
! evaluate_functional_correctness samples_codeqwen_SelfConsistency_pass@1.jsonl

Reading samples...
164it [00:00, 7028.00it/s]
Running test suites...
100%|█████████████████████████████████████████| 164/164 [00:01<00:00, 93.34it/s]
Writing results to samples_codeqwen_SelfConsistency_pass@1.jsonl_results.jsonl...
100%|██████████████████████████████████████| 164/164 [00:00<00:00, 33417.50it/s]
{'pass@1': 0.6951219512195121}


## AST SCOT

In [ ]:
import requests
from collections import Counter

# ================================
# 1️⃣ OLLAMA GENERATOR
# ================================

OLLAMA_URL = "http://127.0.0.1:11434/api/generate"

def ollama_generate(
    prompt: str,
    model: str,
    temperature: float ,
    num_predict: int
) -> str:

    payload = {
        "model": model,
        "prompt": prompt,
        "temperature": temperature,
        "num_predict": num_predict,
        "stream": False
    }

    resp = requests.post(OLLAMA_URL, json=payload, timeout=300)
    resp.raise_for_status()

    return resp.json().get("response", "").strip()


# ================================
# 4️⃣ SCoT REASONING GENERATION
# ================================

def generate_SCoT(prompt: str, model: str) -> str:
    """
    Generate structured reasoning (NO CODE).
    """

    system_msg = (
        "You are a Python problem-solving assistant.\n"
        "Generate a structured solving process.\n"
        "Do NOT write any code.\n"
        "Use steps with sequences, branches, loops.\n"
    )

    one_shot_example = """
            def first_Repeated_Char(str):
                pass

            Input: str: string
            Output: character

            1. Iterate over each character in the string.
            2. Maintain a dictionary to track seen characters.
            3. If a character appears again, return it.
            4. If no repeated character exists, return None.
            """

    full_prompt = (
        system_msg
        + "\n"
        + one_shot_example
        + "\nNow solve the following problem:\n\n"
        + prompt
    )

    return ollama_generate(
        prompt=full_prompt,
        model=model,
        temperature=0.2,
        num_predict=300
    )


# ==========================================
# 5️⃣ SCoT + AST SELF-CONSISTENCY
# ==========================================

# ==========================================
# 5️⃣ SCoT + AST SELF-CONSISTENCY
# ==========================================

def generate_one_completion_SCoT_self_consistency(
    prompt: str,
    model: str,
    num_scot: int = 2,              # ← 2 reasoning traces
    num_code_per_scot: int = 2      # ← 2 code per reasoning
) -> str:
    """
    2 SCoT traces
    2 code generations per SCoT
    Total = 4 generations per problem
    AST majority voting
    """

    all_completions = []

    # ---- Generate multiple reasoning traces (2) ----
    for _ in range(num_scot):
        try:
            scot_reasoning = generate_SCoT(prompt, model)

            # ---- For each reasoning generate 2 code samples ----
            for _ in range(num_code_per_scot):

                raw = ollama_generate(
                    prompt=(
                        "You are a Python coding assistant."
                        "Write correct and complete Python code only."
                        "Do NOT include explanations.\n\n"
                        "Example:"
                        "Problem: Return the sum of all numbers in a list."
                        "Solution:"
                        "def solution(nums):"
                        "    total = 0"
                        "    for n in nums:"
                        "        total += n"
                        "    return total"
                        f"Problem:\n{prompt}\n\n"
                        f"Structured Solving Process:\n{scot_reasoning}"
                    ),
                    model=model,
                    temperature=0.1,
                    num_predict=350
                )

                cleaned = extract_clean_code(raw)
                cleaned = strip_function_def(prompt, cleaned)

                all_completions.append(cleaned)

        except Exception:
            continue

    if not all_completions:
        return "def solution():\n    pass"

    # ---- AST Normalization + Voting ----
    normalized_to_original = {}
    normalized_forms = []

    for code in all_completions:
        try:
            norm = perfect_normalize_code(code)
            if norm == "":
                continue

            normalized_forms.append(norm)
            normalized_to_original.setdefault(norm, []).append(code)

        except Exception:
            continue

    if not normalized_forms:
        return all_completions[0]

    counter = Counter(normalized_forms)
    most_common_norm, _ = counter.most_common(1)[0]

    best_code = normalized_to_original[most_common_norm][0]

    return best_code




In [12]:
from tqdm import tqdm
from human_eval.data import write_jsonl

model_name = "codellama:7b"

samples = []

for task_id in tqdm(problems, desc="Generating SCoT + AST Self-Consistency"):

    best_code = generate_one_completion_SCoT_self_consistency(
        prompt=problems[task_id]["prompt"],
        model=model_name,
        num_scot=5,
        num_code_per_scot=5
    )

    samples.append({
        "task_id": task_id,
        "completion": best_code
    })

output_file = "samples_codellama_SCoT_AST_SC_pass@1.jsonl"
write_jsonl(output_file, samples)

print(f"Saved to {output_file}")

Generating SCoT + AST Self-Consistency:   1%|          | 1/164 [00:58<2:37:41, 58.05s/it]

Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: invalid syntax (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: invalid syntax (<unknown>, line 1)
Error in perfect_normalize_code: invalid syntax (<unknown>, line 1)
Error in perfec

Generating SCoT + AST Self-Consistency:   1%|          | 2/164 [02:00<2:43:19, 60.49s/it]

Error in perfect_normalize_code: unexpected indent (<unknown>, line 2)
Error in perfect_normalize_code: invalid syntax (<unknown>, line 1)
Error in perfect_normalize_code: invalid syntax (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 2)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 2)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in per

Generating SCoT + AST Self-Consistency:   2%|▏         | 3/164 [02:42<2:20:04, 52.20s/it]

...

Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error 

Generating SCoT + AST Self-Consistency:  30%|███       | 50/164 [52:41<1:53:14, 59.60s/it]

Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error in perfect_normalize_code: unexpected indent (<unknown>, line 1)
Error 

Generating SCoT + AST Self-Consistency:  30%|███       | 50/164 [57:08<2:10:16, 68.57s/it]


KeyboardInterrupt: 

In [1]:
! evaluate_functional_correctness samples_codellama_SCoT_AST_SC_pass@1.jsonl

Reading samples...
0it [00:00, ?it/s]
Traceback (most recent call last):
  File "/home/vinayak23597/.local/bin/evaluate_functional_correctness", line 11, in <module>
    load_entry_point('human-eval', 'console_scripts', 'evaluate_functional_correctness')()
  File "/usr/lib/python3/dist-packages/pkg_resources/__init__.py", line 490, in load_entry_point
    return get_distribution(dist).load_entry_point(group, name)
  File "/usr/lib/python3/dist-packages/pkg_resources/__init__.py", line 2854, in load_entry_point
    return ep.load()
  File "/usr/lib/python3/dist-packages/pkg_resources/__init__.py", line 2445, in load
    return self.resolve()
  File "/usr/lib/python3/dist-packages/pkg_resources/__init__.py", line 2451, in resolve
    module = __import__(self.module_name, fromlist=['__name__'], level=0)
  File "/home/vinayak23597/human-eval/human_eval/evaluate_functional_correctness.py", line 28, in <module>
    sys.exit(main())
  File "/home/vinayak23597/human-eval/human_eval/evaluate_fu